In [ ]:
# standard library imports 
from os.path import isfile
from os import remove
import logging

# let gets the version of sqlalchemy
import sqlalchemy
print (sqlalchemy.__version__)

# this helps turn down sqlalchemy logging
logging.basicConfig()
logging.getLogger('sqlalchemy').setLevel(logging.ERROR)

# flip this boolean to cleanup the database to test again
cleanup = True

if cleanup:
    if isfile("movies.db"):
        print ("deleted database file.")
        remove('movies.db')
        

In [ ]:
# if you are going to use sqlalchemy in production I recommend using the C extention here:
# https://docs.sqlalchemy.org/en/13/intro.html#installing-the-c-extensions
# and also using a more perminate backend database (mysql/postgres)
# for demo purposes we are going to use sqllite

from sqlalchemy import Column, Integer, String, create_engine
from sqlalchemy.orm import DeclarativeBase

# db connection
engine = create_engine('sqlite:///movies.db')
# if you want to see lots of output on what sqlalchemy is doing use echo = True
#engine = create_engine('sqlite:///movies.db', echo = True)

# declarative base class
class Base(DeclarativeBase):
    pass

# are class inherits from Base so we get a bunch of new features 
class Movies(Base):
    __tablename__ = 'movies' # name of the table in the db
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    stars = Column(String)
    
    # we are overloading the __repr__ method so we can see this better
    def __repr__(self):
        return "<Movies(name='%s', stars='%s')>" % (self.name, self.stars)

Movies.__table__
Base.metadata.create_all(engine)
print ('created table movies...')

In [ ]:
# using sessions and starting to insert data
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

ins1 = Movies(name="Star Wars", stars="4 stars")
ins2 = Movies(name="Dark Knight", stars="4 stars")

session.add(ins1)
session.add(ins2)
session.commit()

print ("inserted 2 rows...")

In [ ]:
# we can also chain inserts

ins1 = Movies(name="The Big Lebowski", stars="4 stars")
ins2 = Movies(name="Innerspace", stars="3 stars")

session.add_all([ins1,ins2])
session.commit()
print ("inserted 2 rows...")

In [ ]:
# lots of times you need to do an insert and get the id 
# of the row you just inserted.  

ins1 = Movies(name="Vanilla sky", stars="4 stars")
session.add(ins1)
session.commit()
print (ins1.id)

# if you try to get the id before commit you get a None
# so make sure to always commit() before trying retrieving the id
ins1 = Movies(name="Vanilla Sky", stars="4 stars")
session.add(ins1)
print (ins1.id)

In [ ]:
# sqlalchemy supports save points, rollbacks, and transactions
# I don't recommend using these with sqlite so I will just provide
# a couple links to resources

# https://transaction.readthedocs.io/en/latest/sqlalchemy.html
# https://docs.sqlalchemy.org/en/13/orm/tutorial.html#rolling-back

In [ ]:
# select * from table 

for instance in session.query(Movies):
    print(instance.name)
    print (instance.stars)
    print ()

In [ ]:
# select based on order 
for instance in session.query(Movies).order_by('name'):
    print(instance.name)
    print (instance.stars)
    print ()

In [ ]:
# select based on where clause 
for instance in session.query(Movies).filter(Movies.name == "Dark Knight"):
    print(instance.name)
    print (instance.stars)
    print ()

In [ ]:
# select using or
from sqlalchemy import or_

for instance in session.query(Movies).filter(or_(Movies.name == "Dark Knight", Movies.name == "Star Wars")):
    print(instance.name)
    print (instance.stars)
    print ()
    

In [ ]:
# select using and
for instance in session.query(Movies).filter(Movies.name == "Dark Knight").filter(Movies.stars == "4 stars"):
    print(instance.name)
    print (instance.stars)
    print ()

In [ ]:
# select using like
for instance in session.query(Movies).filter(Movies.name.like("%ar%")):
    print(instance.name)
    print (instance.stars)
    print ()

In [ ]:
# you also have a bunch of other properties
print (session.query(Movies).first())

print ()
print (session.query(Movies).all())

print ()
print (session.query(Movies).count())

In [ ]:
# lets delete our table because now we are going to define
# some relationship. We have to close our session or we will
# get a lock using sqlite

from sqlalchemy.exc import OperationalError

session.close()
try:
    Movies.__table__.drop(engine)
except OperationalError:
    print ("table movies does not exist...")
    pass

try:
    Actors.__table__.drop(engine)
except (OperationalError, NameError):
    print ("table actors does not exist...")
    pass

session.close()
engine.dispose()
print ('Table Succesfully dropped...')

In [ ]:
# example of one to many relationship
import os

from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.orm import DeclarativeBase

# declarative base class
class Base(DeclarativeBase):
    pass


def check_file(file):
    if os.path.isfile(file):
        os.remove(file)


class Movies(Base):
    __tablename__ = 'movies'
    __table_args__ = {'extend_existing': True}

    id = Column(Integer, primary_key=True)
    name = Column(String)
    stars = Column(String)
    children = relationship("Actors")


class Actors(Base):
    __tablename__ = 'actors'
    __table_args__ = {'extend_existing': True}

    id = Column(Integer, primary_key=True)
    name = Column(String)
    gender = Column(String)
    age = Column(Integer)
    parent_id = Column(Integer, ForeignKey('movies.id'))


if __name__ == "__main__":
    file = 'db.db'
    check_file(file)
    engine = create_engine('sqlite:///' + file)
    Base.metadata.create_all(engine)

    Session = sessionmaker(bind=engine)
    session = Session()

    # lets define some movies
    movie1 = Movies(name="Vanilla Sky", stars="4 stars")
    movie2 = Movies(name="Star Wars", stars="4 stars")

    actor1 = Actors(name="Tom Cruise", gender="male", age=57)
    actor2 = Actors(name="Penélope Cruz", gender="female", age=45)
    actor3 = Actors(name="Cameron Diaz", gender="female", age=47)
    actor4 = Actors(name="Val Kilmver", gender="male", age=60)
    actor5 = Actors(name="Mark Hamill", gender="male", age=68)
    actor6 = Actors(name="Harrison Ford", gender="male", age=77)

    movie1.children = [actor1, actor2, actor3]
    movie2.children = [actor5, actor6]

    session.add(movie1)
    session.add(movie2)
    session.commit()

    for instance in session.query(Movies).all():
        print(instance.name)
        print(instance.stars)

        print()

        for each in instance.children:
            print(each.name)
            print(each.gender)
            print(each.age)

        print()

    print("script completed...")


In [ ]:
# more documentation about relationships can be found here: 
# https://docs.sqlalchemy.org/en/20/orm/relationships.html

next tutorial: [0018_decorators.ipynb](https://mybinder.org/v2/gh/thesheff17/pythonexamples/master?filepath=src%2F0018_decorators.ipynb)